## Testing end-to-end

In [2]:
import json
import sys
import os
sys.path.append('/Users/gisaac/Documents/GitHub/river_flow_notebooks/')
with open("/Users/gisaac/Documents/GitHub/river_flow_notebooks/01104200.json") as f:
    data = json.load(f)

In [3]:
from scraping_functions import HydroScraper

In [12]:
from datetime import datetime
h = HydroScraper(datetime(2000, 1, 1), datetime(2022, 12, 31), "/Users/gisaac/Documents/GitHub/river_flow_notebooks/01104200.json", "weather_asos_final")

Getting request from USGS
http://waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites=01104200&startDT=2000-01-01&endDT=2022-12-31&parameterCd=00060,00065,00045&siteStatus=all
Request finished


/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:119: DtypeWarning: Columns (1,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep="\t")
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["datetime"] = df["datetime"].map(lambda x: old_timezone.localize(datetime.strptime(x, "%Y-%m-%d %H:%M")).astimezone(new_timezone))
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

Getting request from ASOS
https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=OWD&data=tmpf&data=dwpf&data=relh&data=feel&data=sknt&data=sped&data=alti&data=mslp&data=drct&data=ice_accretion_1hr&data=p01m&data=vsby&data=gust&data=skyc1&data=peak_wind_gust&data=snowdepth&year1=2000&month1=1&day1=1&year2=2023&month2=1&day2=2&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=3&report_type=4


/Users/gisaac/Documents/GitHub/river_flow_notebooks/weather_scraping_functions.py:33: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path) # , parse_dates=['valid']


       station             valid   tmpf   dwpf   relh   feel   sknt   sped  \
0          OWD  2000-01-01 00:53  18.00  13.00  80.29  17.96   0.00   0.00   
1          OWD  2000-01-01 01:53  17.00  13.00  83.47  17.06   0.00   0.00   
2          OWD  2000-01-01 02:53  16.00  13.00  87.47  15.98   0.00   0.00   
3          OWD  2000-01-01 03:53  15.00  13.00  90.96  15.08   0.00   0.00   
4          OWD  2000-01-01 04:53  13.00   7.00  76.68  12.92   0.00   0.00   
...        ...               ...    ...    ...    ...    ...    ...    ...   
272127     OWD  2023-01-01 19:53  52.00  34.00  50.12  52.00  11.00  12.65   
272128     OWD  2023-01-01 20:53  49.00  30.90  49.63  44.53   9.00  10.35   
272129     OWD  2023-01-01 21:53  47.00  30.90  53.50  41.65  10.00  11.50   
272130     OWD  2023-01-01 22:53  46.00  30.90  55.36  41.84   7.00   8.05   
272131     OWD  2023-01-01 23:53  45.00  30.90  57.50  41.83   5.00   5.75   

         alti     mslp    drct ice_accretion_1hr  p01m   vsby  

/Users/gisaac/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


LoadJob<project=hydro-earthnet-db, location=EU, id=7dd449e8-d2da-47d5-bda9-813ae59cd62f>
ASOS data written to BigQuery


In [28]:
import pytz
def combine_data(h) -> None:
    tz = pytz.timezone("UTC")
    if h.asos_df.hour_updated.dt.tz is None:
        h.asos_df['hour_updated'] = h.asos_df['hour_updated'].map(lambda x: x.tz_localize("UTC"))
    joined_df = h.asos_df.merge(h.final_usgs, left_on='hour_updated', right_on='datetime', how='inner')
    nan_precip = sum(pd.isnull(joined_df['p01m']))
    nan_flow = sum(pd.isnull(joined_df['cfs']))
    h.joined_df = joined_df
    h.nan_flow = nan_flow
    h.nan_precip = nan_precip
    h.joined_df = joined_df
    h.joined_df.drop(columns=["site_no"], inplace=True)
    columns_to_drop = [col for col in h.joined_df.columns if col.endswith('_cd')]
    h.joined_df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
h.combine_data()

In [29]:
#combine_data(h)

In [30]:
h.joined_df

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct,sped,alti,...,vsby,gust,skyc1,peak_wind_gust,snowdepth,station_id,index,datetime,cfs,height
0,2000-01-01 05:00:00+00:00,0.00,2000-01-01 04:53,13.0,7.0,NaN,NaN,0.0,0.00,30.18,...,9.0,NaN,CLR,NaN,NaN,OWD,1,2000-01-01 05:00:00+00:00,146.0,NaN
1,2000-01-01 16:00:00+00:00,0.00,2000-01-01 15:53,40.0,31.0,NaN,1024.1,240.0,5.75,30.24,...,8.0,NaN,CLR,NaN,NaN,OWD,45,2000-01-01 16:00:00+00:00,165.0,NaN
2,2000-01-01 17:00:00+00:00,0.00,2000-01-01 16:53,44.0,29.0,NaN,1022.9,240.0,10.35,30.21,...,10.0,NaN,CLR,NaN,NaN,OWD,49,2000-01-01 17:00:00+00:00,165.0,NaN
3,2000-01-01 18:00:00+00:00,0.00,2000-01-01 17:53,46.0,30.0,NaN,1022.0,220.0,9.20,30.18,...,10.0,NaN,CLR,NaN,NaN,OWD,53,2000-01-01 18:00:00+00:00,168.0,NaN
4,2000-01-01 19:00:00+00:00,0.00,2000-01-01 18:53,49.0,30.0,NaN,1021.3,240.0,18.40,30.16,...,10.0,21.0,CLR,NaN,NaN,OWD,57,2000-01-01 19:00:00+00:00,172.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198523,2023-01-01 00:00:00+00:00,1.02,2022-12-31 23:53,56.0,53.0,NaN,1010.4,0.0,0.00,29.84,...,10.0,NaN,OVC,NaN,NaN,OWD,975714,2023-01-01 00:00:00+00:00,498.0,3.89
198524,2023-01-01 01:00:00+00:00,0.25,2023-01-01 00:53,56.0,54.0,NaN,1009.0,190.0,3.45,29.80,...,10.0,NaN,OVC,NaN,NaN,OWD,975726,2023-01-01 01:00:00+00:00,503.0,3.90
198525,2023-01-01 02:00:00+00:00,0.51,2023-01-01 01:53,56.0,53.0,NaN,1008.4,190.0,3.45,29.78,...,9.0,NaN,OVC,NaN,NaN,OWD,975738,2023-01-01 02:00:00+00:00,498.0,3.89
198526,2023-01-01 03:00:00+00:00,2.03,2023-01-01 02:53,56.0,53.0,NaN,1007.3,170.0,4.60,29.75,...,8.0,NaN,OVC,NaN,NaN,OWD,975750,2023-01-01 03:00:00+00:00,503.0,3.90


## Add Sentinel Images

In [44]:
import pandas as pd
h.final_df = h.joined_df

In [32]:
sentinel_df = pd.read_csv("/Users/gisaac/Downloads/sent_data.csv")

In [45]:
h.combine_sentinel(sentinel_df, data['sentinel_2_tile_id'])
h.final_df.drop(columns=["sensing_time"], inplace=True)

In [42]:
h.final_df.drop(columns=["sensing_time"], inplace=True)

KeyError: "['sensing_time'] not found in axis"

In [47]:
#h.final_df.drop(columns=["sensing_time"], inplace=True)
h.write_final_df_to_bq("flow_final_2")

LoadJob<project=hydro-earthnet-db, location=EU, id=c8b7962e-c204-4590-ad7e-3d9a7646f32c>


AttributeError: 'bool' object has no attribute 'result'

In [ ]:
def combine_sentinel(final_df, sentinel_df, tile):
    """ to combine the Sentinel data with the joined ASOS, USGS, and SNOTEL data.
    """
    sentinel_df = sentinel_df[sentinel_df["mgrs_tile"]==tile] 
    sentinel_df = sentinel_df[["sensing_time", "base_url"]]
    sentinel_df["sensing_time"] = pd.to_datetime(sentinel_df["sensing_time"], utc=True, format='mixed').round('60min')
    final_df =final_df.merge(sentinel_df, left_on="hour_updated", right_on="SENSING_TIME", how="left")
    return final_df

fin_df = combine_sentinel(h.final_df, sentinel_df, "11SMT")
fin_df

In [ ]:
fin_df.dropna(subset=["base_url"])

In [ ]:
 sentinel_df_11 = sentinel_df[sentinel_df["mgrs_tile"]=="11SMT"] 

In [ ]:
sentinel_df_11

In [ ]:
find = fin_df

In [ ]:
find.to_csv("final_11059300.csv")

In [ ]:
data

In [ ]:
h.final_df

In [ ]:
def write_final_df_to_bq(h, client, table_name: str) -> bool:
    sample = h.final_df[['hour_updated', 'p01m', 'valid', 'tmpf', 'dwpf', 'ice_accretion_1hr',
       'mslp', 'drct', 'sped', 'alti', 'relh', 'sknt', 'feel', 'vsby', 'gust',
       'skyc1', 'peak_wind_gust', 'snowdepth', 'cfs', 'height']]
    job = client.client.load_table_from_dataframe(sample, table_name)
    print(job.result())
    return True
h.final_df["height"] = pd.to_numeric(h.final_df["height"], errors='coerce')
h.final_df["height"] = pd.to_numeric(h.final_df["height"], errors='coerce')

In [ ]:
write_final_df_to_bq(h, h.bq_connect, "hydronet.flow7")

In [ ]:
h.joined_df.columns

In [ ]:
h.final_df["index"] = h.final_df["index"].astype('str')

In [ ]:
h.final_df["index"]

In [ ]:
write_final_df_to_bq(h, h.bq_connect, "hydronet.flow3")

In [ ]:
h.final_df["station_id"] = h.final_df["station_id"].astype('str')

In [62]:
from google.cloud import storage
client = storage.Client(project="hydro-earthnet-db")
upload_file_to_gcs(client, h.asos_df, "OWI", file_type="weather")

/Users/gisaac/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [58]:
from os import path
def upload_file_to_gcs(client, df, site_no, bucket_name="flow_hydro_2_data", file_type="joined_df"):
    csv_file = df.to_csv()
    bucket = client.get_bucket(bucket_name)
    gcs_path = file_type
    # Define the blob path
    blob = bucket.blob(path.join(gcs_path, site_no + ".csv"))

    # Upload the CSV data to the blob
    blob.upload_from_string(csv_file, content_type='text/csv')

In [49]:
uploa

NameError: name 'uploa' is not defined

In [61]:
h.asos_df

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct,sped,alti,relh,sknt,feel,vsby,gust,skyc1,peak_wind_gust,snowdepth,station_id
0,2000-01-01 01:00:00+00:00,0.0,2000-01-01 00:53,18.0,13.0,NaN,NaN,0.0,0.00,30.14,80.29,0.0,17.96,10.0,NaN,CLR,NaN,NaN,OWD
1,2000-01-01 02:00:00+00:00,0.0,2000-01-01 01:53,17.0,13.0,NaN,NaN,0.0,0.00,30.15,83.47,0.0,17.06,10.0,NaN,CLR,NaN,NaN,OWD
2,2000-01-01 03:00:00+00:00,0.0,2000-01-01 02:53,16.0,13.0,NaN,NaN,0.0,0.00,30.16,87.47,0.0,15.98,10.0,NaN,CLR,NaN,NaN,OWD
3,2000-01-01 04:00:00+00:00,0.0,2000-01-01 03:53,15.0,13.0,NaN,NaN,0.0,0.00,30.17,90.96,0.0,15.08,9.0,NaN,CLR,NaN,NaN,OWD
4,2000-01-01 05:00:00+00:00,0.0,2000-01-01 04:53,13.0,7.0,NaN,NaN,0.0,0.00,30.18,76.68,0.0,12.92,9.0,NaN,CLR,NaN,NaN,OWD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199882,2023-01-01 20:00:00+00:00,0.0,2023-01-01 19:53,52.0,34.0,NaN,1010.3,280.0,12.65,29.84,50.12,11.0,52.00,10.0,NaN,CLR,NaN,NaN,OWD
199883,2023-01-01 21:00:00+00:00,0.0,2023-01-01 20:53,49.0,30.9,NaN,1011.6,300.0,10.35,29.87,49.63,9.0,44.53,10.0,16.0,CLR,NaN,NaN,OWD
199884,2023-01-01 22:00:00+00:00,0.0,2023-01-01 21:53,47.0,30.9,NaN,1012.2,280.0,11.50,29.89,53.50,10.0,41.65,10.0,NaN,CLR,NaN,NaN,OWD
199885,2023-01-01 23:00:00+00:00,0.0,2023-01-01 22:53,46.0,30.9,NaN,1013.1,260.0,8.05,29.92,55.36,7.0,41.84,10.0,NaN,SCT,NaN,NaN,OWD
